<a href="https://colab.research.google.com/github/hjjimmykim/HeadDirectionCells/blob/master/VLearning_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LOLA

Ideas:
1. RL vs. RL (multiagent)
2. Communication channel (pre-set? deception?)
3. LSTM stacked on FWN
4. Induce RL to learn ZD-strategy (learn probabilities?)
5. RL that adapts to opponent's non-stationary strategy

# Libraries

In [27]:
%load_ext autoreload
%autoreload 2

# Standard
import numpy as np
import matplotlib.pyplot as plt
import copy
import time
import sympy as sp
from scipy.optimize import fsolve

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Parameters

In [0]:
#Reward structure
T = 5 # Temptation
R = 3 # Reward
P = 1 # Penalty
S = 0 # Sucker

#LOLA Reward structure
#T = 0 # Temptation
#R = -1 # Reward
#P = -2 # Penalty
#S = -3 # Sucker

# Battle of the sexes
T = 2
R = 0
P = 0
S = 1

# Two Nash Eq., one 

# Reward matrix (0 = cooperate, 1 = defect)
RM = np.zeros([2,2])
RM[0][0] = R
RM[0][1] = S
RM[1][0] = T
RM[1][1] = P

# RL
gamma1 = 0.6 # Decay rate
alpha1 = 0.005 # Learning rate
gamma2 = 0.9 # Decay rate
alpha2 = 0.005 # Learning rate

# Simulation
n_eps = 3       # Training episodes
n_steps = 300      # Training steps
n_turns = 10000       # Turns in the test game
n_rec = 50        # For printing purposes

# Models

In [0]:
# Converts action-pair [a1,a2] into a scalar state (0,1,2,3,4)
def action_to_state(a1,a2):
  
  if a1 == -1 and a2 == -1: # Initial
    s = 0
  elif a1 == 0 and a2 == 0: # CC
    s = 1
  elif a1 == 0 and a2 == 1: # CD
    s = 2
  elif a1 == 1 and a2 == 0: # DC
    s = 3
  elif a1 == 1 and a2 == 1: # DD
    s = 4
    
  return s

In [0]:
def mathematica2py(input = 'V.txt', ID_me=1, ID_otherguy=2):
    f = open(input)

    text = f.read()

    text = ''.join(text.split()) # Remove whitespace
    text = text.replace('^','**') # Replace ^ with **

    num_terms = text.count('{')
    
    # Extract terms and get rid of curly braces
    terms = []
    
    # Scalar-valued
    if text.find(',') == -1:
      return text
    
    # Vector-valued
    while text.find(',') > -1:
      return([x.strip('{').strip('}') for x in text.split(',')])
      cut_point = text.find(',')
      terms.append(text[:cut_point].strip('{').strip('}'))
      text = text[cut_point+1:]
        
    return terms

In [0]:
def evaluate_gradient(dV, a, b, gamma_, gamma2_ = '0'):
  # dV = output of mathematica2py
  # a = my policy
  # b = other guy's policy
  
  a0 = a[0]
  a1 = a[1]
  a2 = a[2]
  a3 = a[3]
  a4 = a[4]
  
  b0 = b[0]
  b1 = b[1]
  b2 = b[2]
  b3 = b[3]
  b4 = b[4]
  
  gamma = gamma_
  gamma2 = gamma2_
  
  return eval(dV)

In [0]:
def find_gamma(dV, a, b):

  gamma = sp.Symbol('gamma',positive=True)

  small = 1e-8
  
  #return sp.nsolve(eval(dV),gamma,(0.9))

  def f1(gamma):
    a0 = a[0]
    a1 = a[1]
    a2 = a[2]
    a3 = a[3]
    a4 = a[4]

    b0 = b[0]
    b1 = b[1]
    b2 = b[2]
    b3 = b[3]
    b4 = b[4]
    
    return eval(dV)
  
  #f1 = lambda gamma: eval(dV)
  
  return fsolve(f1,0.9)

In [33]:
V = mathematica2py('V.txt') # Value function (5 elements for each state)

# Gradients
DVme = [] # w.r.t. my parameters (ai's)
for i in range(5):
  DVme.append(mathematica2py('DVa' + str(i) + '_total.txt'))
LOLA_total = mathematica2py('LOLA_total.txt')
  
'''
DVotherguy = [] # w.r.t. other guy's parameters (bi's)
for i in range(5):
  DVotherguy.append(mathematica2py('DVb' + str(i) + '.txt'))
'''

# Note: state = (player a, player b)

p1 = []
for i in range(5):
  p1.append(np.random.random())
# All-coop
p1 = [1,1,1,1,1]
# All-defect
#p1 = [0,0,0,0,0]
p1=[0.5,0.5,0.5,0.5,0.5]

p2 = []
for i in range(5):
  p2.append(np.random.random())
p2=[1,1,1,1,1]
p2=[0.5,0.5,0.5,0.5,0.5]
# Win-stay-lose-shift
#p2 = [1,1,0,0,1]
# Tit-for-tat
p2 = [1,1,1,0,0]
# ZD-strategy with extortion factor 3
#p2 = [0,11/13,7/26,1/2,0]
#p2 = [0,9/13,7/13,0,0]
#p2 = [0,12/13,7/52,3/4,0]

# DVme[i][j] = Gradient of state j value function with respect to a_i
# DVotherguy[i][j] = Gradient of state j value function with respect to b_i
# second index j does not exist if using total value function (sum over j's)

#print(evaluate_gradient(DVme[0][0], p1, p2))

# p2 stable for high gamma
#p1=[0.26359499,0,0.13090269,0.2684707,0]
#p2=[0,1,0,0,1]

# Stable coop?
p1 = [1,1,0.2,1,0.5]
p2 = [1,1,1,0,0.5]

p1 = [0,0,0,0,0]
p1 = []
for i in range(5):
  p1.append(np.random.random())
p1 = [0.32890504, 0.72718054, 0.29262048, 0.60808543999, 0.120000974]

p1 = [0.5,0.5,0.5,0.5,0.5]
p2 = [0.5,0.5,0.5,0.5,0.5]
print(p1)
print(p2)


[0.5, 0.5, 0.5, 0.5, 0.5]
[0.5, 0.5, 0.5, 0.5, 0.5]


# Game

In [34]:
t_start = time.time()
t1 = time.time()

max_crit_gamma_list1 = []
max_crit_gamma_list2 = []


# Gradient ascent
for i_ep in range(n_steps): 

    # To make updates simultaneous
    p1_old = copy.deepcopy(p1)
    p2_old = copy.deepcopy(p2) 
    
    for i_pol in range(5):
            
      # Naive step for agent 1
      p1[i_pol] = p1[i_pol] + alpha1*evaluate_gradient(DVme[i_pol],p1_old,p2_old,gamma1)      
      
      # LOLA step for agent 1
      #p1[i_pol] = p1[i_pol] + evaluate_gradient(LOLA_total[i_pol],p1_old,p2_old,gamma1,gamma2) * alpha1 * alpha2

      # Naive step for agent 2
      p2_old[2],p2_old[3] = p2_old[3],p2_old[2] # Switch CD & DC
      p1_old[2],p1_old[3] = p1_old[3],p1_old[2]
      p2[i_pol] = p2[i_pol] + alpha2*evaluate_gradient(DVme[i_pol],p2_old,p1_old,gamma2)
      p1_old[2],p1_old[3] = p1_old[3],p1_old[2]
      p2_old[2],p2_old[3] = p2_old[3],p2_old[2]
      
      # Probability constraints
      p1[i_pol] = min(max(0,p1[i_pol]),1)
      p2[i_pol] = min(max(0,p2[i_pol]),1)

    if i_ep % n_rec == 0 and i_ep != 0: 
      t2 = time.time()
      print('-----------------------------')
      print('Episode ' + str(i_ep))
      print(p1)
      print(p2)
      print('Runtime for episodes ' + str(i_ep - n_rec) + '-' + str(i_ep) + ': ' + str(t2-t1) + ' s')
      
      t1 = t2
      
      print('Critical gammas:')
      
      #Find critical gammas
      fg1_list = []
      fg2_list = []
      for i_pol in range(5):
        
        fg1 = find_gamma(DVme[i_pol],p1,p2)
        
        # For nsolve
        if fg1 > 0:
          fg1_list.append(fg1[0])
        else:
          fg1_list.append(np.Inf)
        
        # For solve
        #if len(fg1) > 0:
        #fg1_list.append(fg1)[0])
        #else:
        #  fg1_list.append(np.Inf)
        

        p2[2],p2[3] = p2[3],p2[2] # Switch CD & DC
        p1[2],p1[3] = p1[3],p1[2]
        
        fg2 = find_gamma(DVme[i_pol],p2,p1)
        
        if fg2 > 0:
          fg2_list.append(fg2[0])
        else:
          fg2_list.append(np.Inf)
          
        #if len(fg2) > 0:
        #fg2_list.append(fg2)[0])
        #else:
        #  fg2_list.append(np.Inf)

        p1[2],p1[3] = p1[3],p1[2]
        p2[2],p2[3] = p2[3],p2[2]

        
      print(fg1_list)
      print(fg2_list)
      print('Max. crit. gamma for player 1:',max(fg1_list))
      print('Max. crit. gamma for player 2:',max(fg2_list))
      
      max_crit_gamma_list1.append(max(fg1_list))
      max_crit_gamma_list2.append(max(fg2_list))

      print('-----------------------------')

-----------------------------
Episode 50
[0.449944290162481, 0.5824460223545668, 0.9628584060618461, 0.6442887017831588, 0.8944623393428434]
[0.36445439440037053, 0, 0, 0, 0]
Runtime for episodes 0-50: 17.122862100601196 s
Critical gammas:
[inf, inf, inf, inf, inf]
[inf, inf, inf, inf, inf]
Max. crit. gamma for player 1: inf
Max. crit. gamma for player 2: inf
-----------------------------
-----------------------------
Episode 100
[0.4819921528851481, 0.7452074651252341, 1, 0.8091442476925869, 1]
[0.16004639481024674, 0, 0, 0, 0]
Runtime for episodes 50-100: 24.787660121917725 s
Critical gammas:
[inf, inf, inf, inf, inf]
[inf, inf, inf, inf, inf]
Max. crit. gamma for player 1: inf
Max. crit. gamma for player 2: inf
-----------------------------
-----------------------------
Episode 150
[0.6006817712490349, 0.8983459989818003, 1, 0.9621886960271776, 1]
[0, 0, 0, 0, 0]
Runtime for episodes 100-150: 24.145750522613525 s
Critical gammas:
[inf, inf, inf, inf, inf]
[inf, inf, inf, inf, inf]
M

In [26]:
# Test game
p1 = [1,0,0,0,1]

# Keep track of rewards
rewards1 = np.zeros([n_turns,1])
rewards2 = np.zeros([n_turns,1])

# Keep track of actions
actions1 = np.zeros([n_turns,1])
actions2 = np.zeros([n_turns,1])

# Initial state
s = 0

for i_turn in range(n_turns):

  # Choose action
  # 0 = cooperate, 1 = defect
  a1 = int(np.random.rand(1) > p1[s])
  a2 = int(np.random.rand(1) > p2[s])
  
  actions1[i_turn] = a1
  actions2[i_turn] = a2

  # Get rewards
  r1 = RM[a1][a2]
  r2 = RM[a2][a1]

  rewards1[i_turn] = r1
  rewards2[i_turn] = r2

  # Update the state
  s = action_to_state(a1,a2)

#print(actions1)
#print(actions2)
print(np.sum(rewards1)/n_turns)
print(np.sum(rewards2)/n_turns)
  
print('Total runtime: ' + str(time.time() - t_start) + ' s')

0.5
1.0
Total runtime: 301.60312485694885 s


In [0]:
2.3592/3.7

In [0]:
dVme

In [0]:
if 1 in b:
  b.remove(1)

Abstract

Dynamical Implications of Hyperparameters in Reinforcement Learning

Reinforcement learning has been gaining popularity within the physics research community over past several years. Many of the algorithms require selection of hyperparameters, and it is unclear how these choices affect the learned policy. Often in practice, researchers choose hyperparameters for their simulations using brute force methods such as exhaustive grid search or heuristics without fully understanding how they may alter the effective reward structure and the corresponding value function landscape. Here, we investigate the possibility of using nonlinear dynamics approach to more rigorously guide our selection and adaptive optimization of appropriate parameters that are better aligned to achieve particular aims of the agent. We then extend our investigation to the setting of multiagent reinforcement learning and game theory to see how the choice of agent hyperparameters interplay with each other in competitive, cooperative, and mixed settings.

1) Find toy problem for analytics and motivation

1a) Can use perturbation theory for small gamma?

2) Numerical simulations for gamma trajectories => explanation for why naive learners fail to cooperate?

3) Optimal gamma as a function of turns?

4) Tournament gammas (evolutionary algorithm?)

5) RL controller to adjust gamma in real time? Meta RL controlling naive learner.